# Notebook 2: Table Creation
Create Unity Catalog-managed Delta tables using corrected SQL syntax.

%md
# 📘 Delta Table Creation: Unity Catalog Setup

This notebook sets up all required Delta tables for the Aircraft AOG Demo inside Unity Catalog.

## ✅ Tables Created

1. **`raw_sensor_data`**  
   Stores synthetic sensor readings from aircraft systems.

2. **`maintenance_events`**  
   Stores aircraft maintenance events (e.g., routine checks, engine repairs).

3. **`cleaned_sensor_data`**  
   Created by DLT pipeline. Contains filtered, validated sensor data.

4. **`prediction_results`**  
   Created by model pipeline. Stores risk scores for each aircraft.

> All tables are created in:  
> `CATALOG: arao`, `SCHEMA: aerodemo`

In [0]:
%sql
-- Set catalog and schema
USE CATALOG arao;
USE SCHEMA aerodemo;

-- -- Drop base/raw tables
-- DROP TABLE IF EXISTS raw_sensor_data;
-- DROP TABLE IF EXISTS maintenance_events;

-- Drop intermediate/processed tables
DROP TABLE IF EXISTS cleaned_sensor_data;
DROP TABLE IF EXISTS enriched_sensor_data;
DROP TABLE IF EXISTS sensor_features;

-- Drop ML outputs and reference data
DROP TABLE IF EXISTS prediction_results;
DROP TABLE IF EXISTS anomaly_alerts;
DROP TABLE IF EXISTS aircraft_model_reference;

In [0]:
%sql

-- Set correct catalog and schema
USE CATALOG arao;
USE SCHEMA aerodemo;

-- Create aircraft model reference table for Digital Twin context
CREATE TABLE IF NOT EXISTS aircraft_model_reference (
  model STRING,
  manufacturer STRING,
  engine_type STRING,
  capacity INT,
  range_km INT
)
USING DELTA
COMMENT 'Reference data for aircraft models, used in Digital Twin tracking';


-- Create raw sensor data table
CREATE OR REPLACE TABLE raw_sensor_data (
  timestamp TIMESTAMP,
  aircraft_id STRING,
  model STRING,
  engine_temp DOUBLE,
  fuel_efficiency DOUBLE,
  vibration DOUBLE,
  altitude DOUBLE,
  airspeed DOUBLE,
  anomaly_score DOUBLE,
  oil_pressure DOUBLE,       -- New: measured in psi
  engine_rpm INT,            -- New: revolutions per minute
  battery_voltage DOUBLE     -- New: volts
)
USING DELTA
COMMENT 'Enhanced sensor data with additional flight and engine metrics';

-- Create maintenance events table
CREATE TABLE IF NOT EXISTS maintenance_events (
  aircraft_id STRING,
  event_date DATE,
  event_type STRING
)
USING DELTA
COMMENT 'Maintenance logs per aircraft';

-- Cleaned sensor data table (DLT will populate)
CREATE TABLE IF NOT EXISTS cleaned_sensor_data (
  timestamp TIMESTAMP,
  aircraft_id STRING,
  model STRING,
  engine_temp DOUBLE,
  fuel_efficiency DOUBLE,
  vibration DOUBLE,
  altitude DOUBLE,
  airspeed DOUBLE,
  anomaly_score DOUBLE
)
USING DELTA
COMMENT 'Cleaned sensor data after applying quality checks';

-- Prediction output table (DLT will populate)
CREATE TABLE IF NOT EXISTS prediction_results (
  aircraft_id STRING,
  prediction_date DATE,
  risk_score DOUBLE
)
USING DELTA
COMMENT 'Predicted AOG risk scores from model';



CREATE TABLE IF NOT EXISTS arao.aerodemo.anomaly_alerts (
  aircraft_id STRING,
  prediction_date DATE,
  predicted_anomaly INT
)
USING DELTA
COMMENT 'High-risk anomaly alerts from model predictions';
